# UNIVERSIDADE FEDERAL DE UBERLÂNDIA

# Dinâmica de Rotores

Aluno:
    
    Cristofer Antoni Souza Costa
    
Professor:

    Aldemir Cavallini Jr.

# Sumário

- [Introdução](#Introdução)

- [Importando bibliotecas e efetuando configurações](#Importando-bibliotecas-e-efetuando-configurações)

- [Características do Conjunto Rotativo](#Características-do-Conjunto-Rotativo)

    - [Sistema de coordenadas](#Sistema-de-coordenadas)

    - [Disco](#Disco)

    - [Eixo](#Eixo)



# Introdução

O objetivo desse trabalho consiste em caracterizar o comportamento dinâmico de um sistema formado por um conjunto rotativo submetido a torção e flexão. O conjunto rotativo é composto por um eixo e disco (desbalanceado), suportado por um par de mancais.  Deseja-se prever os seguintes aspectos ainda em etapa de projeto:
1. Torque nominal:
	- Determinação do mínimo diâmetro do eixo.
2. Torção:
	- Compartamento estático e dinamico.
    - Velocidades críticas quanto ao comportamento vibracional torcional.
	- Comportamento em transientes de partida e parada do equipamento.
3. Flexão:
    - Coportamento dinâmico;
	- Velocidades críticas quanto ao comportamento vibracional lateral.
	- Possíveis instabilidades.
	- Resposta à aplicação de massa de desbalanço.
	- Resposta a forças assíncronas.

# Importando bibliotecas e efetuando configurações

In [2]:
import numpy as np
import plotly.graph_objects as go
from pint import UnitRegistry
from plot_vectors import plot_vectors

# Configuring Pint
ureg = UnitRegistry()
Q_ = ureg.Quantity

# Características do Conjunto Rotativo

Os elementos básicos que compôe o conjunto rotativo são: o disco; o eixo; os mancais; e os elementos de selagem. Visto que em aplicações reais, o desblancemento não pode ser completamente eliminado, massas de desbalanceamento também estão presente na modelagem.

Procedimento de modelagem:

1. Determinar expressões para a energia cinética do disco, do eixo e das massas de dos balanços.

2. Determinar expressões para a energia de deformação, com intuito de caracterizar o eixo.

3. Forças para caracterizar mancais e selagens são utilizadas para determinar o trabalho virtual.

## Sistema de coordenadas

- Referencial inercial, sua origem coincide com centro geométrico dos mancais: $R_0:\{O_0, X, Y, Z\}$; e

- Referencial móvel, centro geométrico do disco: $R:\{O,x,y,z\}$.

<div>
<img src="img/Coordinate_system.png" width="400"/>
</div>


### Transfomação de Coordenadas

O fato de o sistema de coordenadas móvel girar, implica que os sistemas móvel $xyz$ e o sistema inercial $XYZ$ passam a guardar uma relação dada por meio dos angulos $\psi=\psi(t)$, $\theta=\theta(t)$ e $\phi=\phi(t)$ conforme figura abaixo. Lembrando que a convenção ora adotada é a regra da mão direita.

<div>
<img src="img/Coordinate_system_ complete.png" width="500"/>
</div>

Para coincidir a orientação do sistema de coordenadas móvel com o sistema inercial: primeiramente o sistema inercial é rotacionado com o angulo $\psi$ em torno do eixo $Z$, originando um novo eixo $x_1$ e $y_1$ (a partir de $X$ e $Y$); posteriormente a rotação $\theta$ se dá em torno do eixo $x_1$, gerando $z_1$ e $y$ (a partir de $y_1$ e $Z$) e por fim a rotação $\phi$ se dá em torno do eixo $y$, gerando $z$ e $x$ (a partir de $z_1$ e $x_1$). 

Assim sendo, se tivermos o vetor 

$$\vec{a}_{R_0} = 
\begin{align}
    \begin{bmatrix}
        0 \\
        1 \\
        0
    \end{bmatrix}_{R_0}
\end{align}
$$

representado utilizando-se os vetores da base $R_0$, aplicando-se a matriz de transformação 
 
$${}_{R_1}T_{R_0} = 
\begin{vmatrix}
    \cos{\psi} & \sin{\psi} & 0 \\ 
    -\sin{\psi} & \cos{\psi} & 0 \\ 
    0 & 0 & 1
\end{vmatrix}$$

a este vetor podemos obter um novo vetor representado na base $R_1 : \{O, x_1, y_1, Z\}$. Considerando-se um ângulo $\psi =\pi / 4$, este novo vetor $\vec{a}_{R_1}$ apresentaria os seguintes valores: 
 
$$\vec{a}_{R_1} = 
\begin{align}
    \begin{bmatrix}
        \frac{\sqrt{2}}{2} \\
        \frac{\sqrt{2}}{2} \\
        0
    \end{bmatrix}_{R_1}
\end{align}
$$

Assim sendo teríamos a aplicação da Matriz de Transformação de Coordenadas do sistema inercial $R_0$ para o sistema móvel $R_1$

$$\vec{a}_{R_1} = {}_{R_1}T_{R_0} \cdot \vec{a}_{R_0} $$

e a sua transposta 

$$\vec{a}_{R_0} = {}_{R_1}T_{R_0}^T \cdot \vec{a}_{R_1} $$

atuaria na transformação de coordenadas do sistema sistema móvel $R_1$ para o sistema inercial $R_0$.

O código a seguir ilustra essas transformações

In [11]:
# Giro em torno de Z - Rotação do disco
psi = np.pi / 4

# Matriz de Transformação de Coordenadas R0_Rz
transformation_R0_R1 = np.array(([np.cos(psi), np.sin(psi),0],
                                 [-np.sin(psi), np.cos(psi), 0],
                                 [0, 0, 1]))

# Verificando a implementação
vector_R0 = np.array(([0, 1, 0]))
vector_R1 = transformation_R0_R1 @ vector_R0
vector_R1

array([0.70710678, 0.70710678, 0.        ])

In [12]:
# o módulo do vector_R é igual ao vector_Rz?
np.linalg.norm(vector_R0) == np.linalg.norm(vector_R1) == 1.0

True

In [13]:
# E voltando para o espaço vetorial RZ para R0
# por intermédio da transposição da matriz de transformação 
transformation_R0_R1.transpose() @ vector_R1

array([4.26642159e-17, 1.00000000e+00, 0.00000000e+00])

### Computando os demais giros e matrizes

Procedendo da mesma forma, efetuando rotação nos eixos $x_1$ e $y$, teríamos:

In [14]:
# Giro em torno de X
theta = np.pi / 4

# Giro em torno de Y
phi = np.pi / 4

transformation_R2_R1 = np.array(([np.cos(psi), np.sin(psi),0],
                                [-np.sin(psi), np.cos(psi), 0],
                                [0, 0, 1]))

transformation_R_R2 = np.array(([np.cos(psi), np.sin(psi),0],
                                [-np.sin(psi), np.cos(psi), 0],
                                [0, 0, 1]))

In [15]:
# Verificando a implementação
vector_R0 = np.array(([0, 1, 0]))
vector_R1 = transformation_R0_R1 @ vector_R0
vector_R1

array([0.70710678, 0.70710678, 0.        ])

In [10]:
vector_R0

array([0, 1, 0])

In [ ]:
v1 = np.array([1],[2],[3])
v1.shape

In [ ]:
column = v1[:, np.newaxis]
column

In [ ]:
val = np.c_[[1,2,3]]

In [ ]:
val @ np.array([1,2,3])

In [ ]:
v1 @ column

In [ ]:
# De r_0 para r_1
vec_2 = transform_01 @ vec_0
vec_2

In [ ]:
# De r_1 para r_1
transform_01.transpose() @ vec_2

In [ ]:


def disk_transformation(vector, psi, theta, phi):
    # Transformation matrix
    # Matrizes de tranformação de coordenadas
    # 
    
    
    
    return vector

v1_R0 = np.array([[1],[1],[1]])

## Disco

Assume-se que o mesmo é rígido e caracterizado somente por sua energia cinética.

### Energia Cinética

## 





In [ ]:
v1_R0.shape

In [ ]:
A = np.array(([1,2],[1,0]))

In [ ]:
A @ np.array(([1,2]))

In [ ]:
# Plotting

shaft = 30

# Cylinder parameters
offset = 15 # from Y origin
r = 2 # radius
h = 1  # height
theta = np.linspace(0, 2*np.pi, 10)
z = r * np.cos(theta)
x = r * np.sin(theta)

# Create cylinder surface
cylinder = go.Surface(x=np.outer(x, np.ones_like(theta)),
                      y=offset + np.outer(np.ones_like(theta), np.linspace(0, h, 50)),
                      z=np.outer(z, np.ones_like(theta)),
                      colorscale='Viridis',
                      showscale=False)

# Line in origin
bearing_line = go.Scatter3d(x=[0, 0],
                            y=[0, shaft],
                            z=[0, 0],
                            mode='lines',
                            marker={'color': 'green'},
                            line={'dash': 'longdash',
                                  'width': 5})

# Create figure
fig = go.Figure(data=[cylinder, bearing_line])

# Set layout
fig.update_layout(scene=dict(aspectratio=dict(x=1, y=1, z=1),
                             aspectmode='manual'))

# Show plot
fig.show()


In [ ]:
np.identity(4)